In [1]:
import os
import pandas as pd
import numpy as np
import math
from pathlib import Path
import geopandas as gpd
import sys
sys.path.append("..") # Adds higher directory to python modules path.
from data_loaders import *
import pickle
import plotly.graph_objects as go
from scipy.stats import gaussian_kde
prefix = '../'

def round_if_float(input, rounding = 1):
    # try integer
    try:
        int_value = int(input)
        if str(int_value) == str(input).strip():
            return int_value
    except (ValueError, TypeError):
        pass
    # alternatively try float
    try:
        return round(float(input), rounding)
    except (ValueError, TypeError):
        pass
    return input

# Preparing National Choropleth Map Data

## Get GEOIDs

In [2]:
# Load State Abbreviations to FIP Codes
states_to_fip_df = pd.read_csv('../data/state_abbreviations_and_fip_codes.csv')
states_to_fip_dict = dict(zip(states_to_fip_df['State_Abbreviation'], states_to_fip_df['FIP_Code']))
state_fips_of_interest = list(states_to_fip_dict.values())
fips_to_state_dict = { code: abbr for abbr, code in states_to_fip_dict.items() }
# I manually removed Puerto Rico and Virgin Islands

# Load and filter the shapefile
# gdf = gpd.read_file('../data/state_shapes_5m/cb_2023_us_state_5m.shp')
gdf = gpd.read_file('../data/state_shapes_20m/cb_2023_us_state_20m.shp')
gdf['STATEFP'] = gdf['STATEFP'].astype(int)
gdf_states = gdf[gdf['STATEFP'].isin(state_fips_of_interest)]
gdf_states.reset_index(inplace = True)
gdf_states['State_Abbreviation'] = gdf_states['STATEFP'].map(fips_to_state_dict)
gdf_states['GEOID'] = gdf_states['GEOID'].astype(str)
gdf_states.rename(columns = {'NAME' : 'State'}, inplace = True)
gdf_states = gdf_states[['State', 'State_Abbreviation', 'GEOID', 'geometry']]

gdf_states.head()

/opt/homebrew/Caskroom/miniforge/base/envs/erdos_fall_2024/lib/python3.12/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/opt/homebrew/Caskroom/miniforge/base/envs/erdos_fall_2024/lib/python3.12/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/var/folders/jk/npp70g_j2fbg8x9prxslv1zw6_ptrw/T/ipykernel_12011/739237553.py:16: SettingWithCopyWarning: 
A value i

,State,State_Abbreviation,GEOID,geometry
0,Texas,TX,48,"POLYGON ((-106.62344 31.91403, -106.63011 31.9..."
1,California,CA,06,"MULTIPOLYGON (((-118.59397 33.4672, -118.48478..."
2,Kentucky,KY,21,"POLYGON ((-89.54443 36.57451, -89.47935 36.566..."
3,Georgia,GA,13,"POLYGON ((-85.60516 34.98468, -85.47434 34.983..."
4,Wisconsin,WI,55,"MULTIPOLYGON (((-86.93428 45.42115, -86.83575 ..."


## Get AP Performance, Availability, and Participation Data

In [3]:
states_abbrv = pd.read_csv(prefix + 'data/State Abbreviation.csv')

# AP Outcomes
AP_outcomes = gimmeOutcomes(year = 2022, prefix = prefix)
AP_outcomes = AP_outcomes[AP_outcomes['Race'] == 'Overall']
AP_outcomes.reset_index(inplace = True)
AP_outcomes['PassRate'] = (AP_outcomes['Score 3'] + AP_outcomes['Score 4'] + AP_outcomes['Score 5']) / AP_outcomes['Total'] * 100
AP_outcomes = AP_outcomes[['State', 'stabbr', 'PassRate', 'Total', 'Mean', 'Score 5', 'Score 4', 'Score 3', 'Score 2', 'Score 1']]

# AP Participation
participation = pd.read_excel(prefix + 'data/AP_data_fixed-participation.xlsx',sheet_name = '2022').transpose()
participation.reset_index(inplace=True)
participation.rename(columns={'index': 'State'}, inplace = True)
participation['i'] = [i for i in range(0, len(participation))]
participation.set_index('i', inplace = True)
participation.columns = participation.iloc[0]
participation = participation[1:]
participation.replace('>99%', '0.99', inplace = True)
participation.replace('**', None, inplace = True)
participation['% Asian'] = participation['% Asian'].astype(float)
participation['% Hispanic/Latino'] = participation['% Hispanic/Latino'].astype(float)
participation['% White'] = participation['% White'].astype(float)
participation['% Black or African American'] = participation['% Black or African American'].astype(float)
participation['% Native American or Alaska Native'] = participation['% Native American or Alaska Native'].astype(float)
participation['% Native Hawaiian or other Pacific Islander'] = participation['% Native Hawaiian or other Pacific Islander'].astype(float)
participation['% Two or More Races'] = participation['% Two or More Races'].astype(float)
participation['% Asian'] *= 100
participation['% Hispanic/Latino'] *= 100
participation['% White'] *= 100
participation['% Black or African American'] *= 100
participation['% Native American or Alaska Native'] *= 100
participation['% Native Hawaiian or other Pacific Islander'] *= 100
participation['% Two or More Races'] *= 100

AP_participation = participation

# AP Availability
availability = pd.read_excel(prefix + 'data/AP_data_fixed-availability.xlsx', sheet_name = '2022').transpose()
availability.reset_index(inplace=True)
availability.rename(columns={'index': 'State'}, inplace = True)
availability['i'] = [i for i in range(0, len(availability))]
availability.set_index('i', inplace = True)
availability.columns = availability.iloc[0]
availability = availability[1:]
AP_availability = availability

print('AP Outcomes')
print(AP_outcomes.columns)
print(' ')
print('AP Participation')
print(AP_participation.columns)
print(' ')
print('AP Availability')
print(AP_availability.columns)


AP Outcomes
Index(['State', 'stabbr', 'PassRate', 'Total', 'Mean', 'Score 5', 'Score 4',
       'Score 3', 'Score 2', 'Score 1'],
      dtype='object')
 
AP Participation
Index(['State', '% Asian', '% Hispanic/Latino', '% White',
       '% Black or African American', '% Native American or Alaska Native',
       '% Native Hawaiian or other Pacific Islander', '% Two or More Races',
       '% No Response', '# Asian', '# Hispanic/Latino', '# White',
       '# Black or African American', '# Native American or Alaska Native',
       '# Native Hawaiian or other Pacific Islander', '# Two or More Races',
       '# No Response'],
      dtype='object', name=0)
 
AP Availability
Index(['State', '5+ Exams #', '5+ Exams %', '5+ Exams Public Offering %',
       '5+ Exams % Asian', '5+ Exams % Hispanic/Latino', '5+ Exams % White',
       '5+ Exams % Black or African American',
       '5+ Exams % Native American or Alaska Native',
       '5+ Exams % Native Hawaiian or other Pacific Islander',
       '5

/var/folders/jk/npp70g_j2fbg8x9prxslv1zw6_ptrw/T/ipykernel_12011/1671719238.py:18: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  participation.replace('>99%', '0.99', inplace = True)


In [4]:
# Add in any columns of interest across these AP dataframes
skeleton = gdf_states.copy()
skeleton['PassRate'] = None
skeleton['Mean'] = None
skeleton['Total'] = None
skeleton['5+Exams%'] = None
skeleton['%Asian'] = None
skeleton['%HispanicOrLatino'] = None
skeleton['%White'] = None
skeleton['%BlackOrAfricanAmerican'] = None
skeleton['%NativeAmericanOrAlaskaNative'] = None
skeleton['%NativeHawaiianOrOtherPacificIslander'] = None
skeleton['%TwoOrMoreRaces'] = None

### Merge AP Outcome Data

In [5]:
merged = skeleton.merge(
    AP_outcomes[['PassRate', 'Mean', 'Total', 'State']],
    on = ['State'],
    how = 'left',
    suffixes = ('', '_')
)

merged['PassRate'] = merged['PassRate'].fillna(merged['PassRate_'])
merged['Mean'] = merged['Mean'].fillna(merged['Mean_'])
merged['Total'] = merged['Total'].fillna(merged['Total_'])

merged = merged.drop(columns=['PassRate_', 'Mean_', 'Total_'])
skeleton = merged

/var/folders/jk/npp70g_j2fbg8x9prxslv1zw6_ptrw/T/ipykernel_12011/2967106224.py:8: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged['PassRate'] = merged['PassRate'].fillna(merged['PassRate_'])
/var/folders/jk/npp70g_j2fbg8x9prxslv1zw6_ptrw/T/ipykernel_12011/2967106224.py:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged['Mean'] = merged['Mean'].fillna(merged['Mean_'])
/var/folders/jk/npp70g_j2fbg8x9prxslv1zw6_ptrw/T/ipykernel_12011/2967106224.py:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will 

### Merge AP Availability Data

In [6]:
merged = skeleton.merge(
    AP_availability[['5+ Exams %', 'State']],
    on = ['State'],
    how = 'left',
    suffixes = ('', '_')
)
merged['5+Exams%'] = merged['5+Exams%'].fillna(merged['5+ Exams %'])
merged['5+Exams%'] *= 100

merged = merged.drop(columns=['5+ Exams %'])
skeleton = merged

/var/folders/jk/npp70g_j2fbg8x9prxslv1zw6_ptrw/T/ipykernel_12011/3656781475.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged['5+Exams%'] = merged['5+Exams%'].fillna(merged['5+ Exams %'])


### Merge AP Participation Data

In [7]:
merged = skeleton.merge(
    AP_participation[['% Asian', '% Hispanic/Latino', '% White', '% Black or African American', '% Native American or Alaska Native', '% Native Hawaiian or other Pacific Islander', '% Two or More Races', 'State']],
    on = ['State'],
    how = 'left',
    suffixes = ('', '_')
)

merged['%Asian'] = merged['%Asian'].fillna(merged['% Asian'])
merged['%HispanicOrLatino'] = merged['%HispanicOrLatino'].fillna(merged['% Hispanic/Latino'])
merged['%White'] = merged['%White'].fillna(merged['% White'])
merged['%BlackOrAfricanAmerican'] = merged['%BlackOrAfricanAmerican'].fillna(merged['% Black or African American'])
merged['%NativeAmericanOrAlaskaNative'] = merged['%NativeAmericanOrAlaskaNative'].fillna(merged['% Native American or Alaska Native'])
merged['%NativeHawaiianOrOtherPacificIslander'] = merged['%NativeHawaiianOrOtherPacificIslander'].fillna(merged['% Native Hawaiian or other Pacific Islander'])
merged['%TwoOrMoreRaces'] = merged['%TwoOrMoreRaces'].fillna(merged['% Two or More Races'])

merged = merged.drop(columns=['% Asian', '% Hispanic/Latino', '% White', '% Black or African American', '% Native American or Alaska Native', '% Native Hawaiian or other Pacific Islander', '% Two or More Races'])
skeleton = merged

/var/folders/jk/npp70g_j2fbg8x9prxslv1zw6_ptrw/T/ipykernel_12011/685087468.py:8: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged['%Asian'] = merged['%Asian'].fillna(merged['% Asian'])
/var/folders/jk/npp70g_j2fbg8x9prxslv1zw6_ptrw/T/ipykernel_12011/685087468.py:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged['%HispanicOrLatino'] = merged['%HispanicOrLatino'].fillna(merged['% Hispanic/Latino'])
/var/folders/jk/npp70g_j2fbg8x9prxslv1zw6_ptrw/T/ipykernel_12011/685087468.py:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .

## Clean up the skeleton

In [8]:
skeleton.replace('**', None, inplace = True)

In [9]:
# In the end, save as CSV for caching
skeleton.to_csv('US_States_Map_Data.csv')

In [10]:
states_of_interest_df = skeleton[skeleton['State_Abbreviation'].isin(['MA', 'WI', 'GA', 'NC'])]
rename_national_features = {
    'PassRate': 'AP Pass Rate (3 or higher)', 
    'Mean': 'AP Score Mean (out of 5)', 
    'Total': 'Total No. AP Exams', 
    '5+Exams%': 'Offer 5+ Exams (%)', 
    '%Asian': 'Asian Participation (%)', 
    '%HispanicOrLatino': 'Hispanic or Latino Participation (%)', 
    '%White': 'White Participation (%)', 
    '%BlackOrAfricanAmerican': 'Black or African American Participation (%)', 
    '%NativeAmericanOrAlaskaNative': 'Native American or Alaska Native Participation (%)', 
    '%NativeHawaiianOrOtherPacificIslander': 'Native Hawaiian or other Pacific Islander Participation (%)', 
    '%TwoOrMoreRaces': 'Two or More Races Participation (%)'
}
states_of_interest_df.rename(columns = rename_national_features, inplace = True, errors = 'raise')
states_of_interest_df.drop(['geometry', 'State', 'GEOID'], inplace = True, axis = 1)

MA_AP_table = states_of_interest_df[states_of_interest_df['State_Abbreviation'] == 'MA']
MA_AP_table.drop(['State_Abbreviation'], inplace = True, axis = 1)
MA_AP_table = MA_AP_table.T
MA_AP_table.rename(columns = {24: 'Value'}, inplace = True)
MA_AP_table['Value'] = MA_AP_table.apply(lambda row : round_if_float(row['Value']), axis = 1)
MA_AP_table.to_csv('MA_AP_table.csv', index = True, index_label = 'Category')

WI_AP_table = states_of_interest_df[states_of_interest_df['State_Abbreviation'] == 'WI']
WI_AP_table.drop(['State_Abbreviation'], inplace = True, axis = 1)
WI_AP_table = WI_AP_table.T
WI_AP_table.rename(columns = {4: 'Value'}, inplace = True)
WI_AP_table['Value'] = WI_AP_table.apply(lambda row : round_if_float(row['Value']), axis = 1)
WI_AP_table.to_csv('WI_AP_table.csv', index = True, index_label = 'Category')

GA_AP_table = states_of_interest_df[states_of_interest_df['State_Abbreviation'] == 'GA']
GA_AP_table.drop(['State_Abbreviation'], inplace = True, axis = 1)
GA_AP_table = GA_AP_table.T
GA_AP_table.rename(columns = {3: 'Value'}, inplace = True)
GA_AP_table['Value'] = GA_AP_table.apply(lambda row : round_if_float(row['Value']), axis = 1)
GA_AP_table.to_csv('GA_AP_table.csv', index = True, index_label = 'Category')

NC_AP_table = states_of_interest_df[states_of_interest_df['State_Abbreviation'] == 'NC']
NC_AP_table.drop(['State_Abbreviation'], inplace = True, axis = 1)
NC_AP_table = NC_AP_table.T
NC_AP_table.rename(columns = {30: 'Value'}, inplace = True)
NC_AP_table['Value'] = NC_AP_table.apply(lambda row : round_if_float(row['Value']), axis = 1)
NC_AP_table.to_csv('NC_AP_table.csv', index = True, index_label = 'Category')

/var/folders/jk/npp70g_j2fbg8x9prxslv1zw6_ptrw/T/ipykernel_12011/3599817936.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  states_of_interest_df.rename(columns = rename_national_features, inplace = True, errors = 'raise')
/var/folders/jk/npp70g_j2fbg8x9prxslv1zw6_ptrw/T/ipykernel_12011/3599817936.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  states_of_interest_df.drop(['geometry', 'State', 'GEOID'], inplace = True, axis = 1)
/var/folders/jk/npp70g_j2fbg8x9prxslv1zw6_ptrw/T/ipykernel_12011/3599817936.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the c

------------
------------

# Creating Small Scoring Distributions for States

In [20]:
# AP Outcomes
AP_outcomes = gimmeOutcomes(year = 2022, prefix = prefix)
AP_outcomes = AP_outcomes[AP_outcomes['Race'] == 'Overall']
AP_outcomes.reset_index(inplace = True)

keep = ['stabbr', 'Total', 'Score 1', 'Score 2', 'Score 3', 'Score 4', 'Score 5']
for i in range(1, 6):
    AP_outcomes[f'Percent {i}'] = AP_outcomes[f'Score {i}'] / AP_outcomes['Total'] * 100
    AP_outcomes[f'Percent {i}'] = AP_outcomes.apply(lambda row : round_if_float(row[f'Percent {i}']), axis = 1)
    keep.append(f'Percent {i}')

AP_outcomes = AP_outcomes[keep]

states_abbrv = pd.read_csv(prefix + 'data/State Abbreviation.csv')
states_dict = dict(zip(states_abbrv['stabbr'], states_abbrv['State']))

In [21]:
AP_outcomes['PassRate'] = (AP_outcomes['Score 3'] + AP_outcomes['Score 4'] + AP_outcomes['Score 5']) / AP_outcomes['Total'] * 100
AP_outcome_reorded = AP_outcomes.sort_values('PassRate')
AP_outcome_reorded

,stabbr,Total,Score 1,Score 2,Score 3,Score 4,Score 5,Percent 1,Percent 2,Percent 3,Percent 4,Percent 5,PassRate
25,MS,14788,5850,3365,2880,1753,940,39.6,22.8,19.5,11.9,6.4,37.685962
19,LA,30434,10890,7248,6086,4039,2171,35.8,23.8,20.0,13.3,7.1,40.402182
32,NM,14328,4951,3571,2982,1761,1063,34.6,24.9,20.8,12.3,7.4,40.522055
4,AR,41105,12925,10841,9253,5533,2553,31.4,26.4,22.5,13.5,6.2,42.182216
9,DC,6840,2547,1403,1119,965,806,37.2,20.5,16.4,14.1,11.8,42.251462
1,AL,44241,13031,11317,9838,6516,3539,29.5,25.6,22.2,14.7,8.0,44.965078
49,WV,9091,2307,2372,2163,1480,769,25.4,26.1,23.8,16.3,8.5,48.531515
44,TX,545526,160842,117958,116559,90096,60071,29.5,21.6,21.4,16.5,11.0,48.893362
29,NV,29932,7811,7042,7180,4982,2917,26.1,23.5,24.0,16.6,9.7,50.377522
37,OK,21467,4870,5707,5364,3650,1876,22.7,26.6,25.0,17.0,8.7,50.729026


In [24]:
def produce_distribution(stabbr):
    df = AP_outcomes[AP_outcomes['stabbr'] == stabbr]
    bins = [1, 2, 3, 4, 5]
    weights = list(df.iloc[0,7:13])

    # KDE curve calculation
    # kde = gaussian_kde(bins, weights = weights)
    # x_kde = np.linspace(min(bins) - 0.5, max(bins) + 0.5, 100)
    # y_kde = kde(x_kde)

    # Create bar plot
    fig = go.Figure()

    colors = ['#FFCC00', '#FF9900', '#FF6600', '#FF3300', '#CC0000']  # Burnt yellow to red
    fig.add_trace(go.Bar(
        x=bins,
        y=weights,
        width=0.5,  # Thinner bars
        marker=dict(color=colors),
        hovertemplate="<b>Score %{x}</b><br><b>Percentage:</b> %{y}%<extra></extra>", 
        showlegend=False
    ))

    # # Add KDE curve trace
    # fig.add_trace(go.Scatter(
    #     x=x_kde,
    #     y=y_kde * 100,  # Adjust scaling to match bar height
    #     mode='lines',
    #     line=dict(color='red', width=2),
    #     showlegend=False
    # ))

    # Update layout for aesthetics
    fig.update_layout(
        title=f"Score Distribution for {states_dict[stabbr]}",
        xaxis_title="AP Exam Score",
        yaxis_title="Percentage",
        barmode='overlay',
        template='plotly_white'
    )

    fig.show()

    # pickle everything
    with open(f'../data/{stabbr}_pickled/{stabbr}_score_distribution.pkl', 'wb') as f:
        pickle.dump(fig, f)

In [25]:
states_of_interest = ['MA', 'WI', 'GA', 'NC']

for stabbr in states_of_interest:
    produce_distribution(stabbr)

--------
-------

# Preparing County-Level Choropleth Map Data

To streamline the Streamlit app's computations, we prepare the static data involved in the choropleth map. This involves collecting all that we know about the AP performances, Income, and Population for each county in Georgia, Massachusetts, and Wisconsin, as well as the shape data so we may draw those counties.

# The Order of Building the Choropleth Data
0. Collect all counties and years under consideration
1. Set up skeleton: all combinations of counties and years
2. Gather all AP performance data per state
3. Fill in the skeleton with the performance data for all three states individually
4. Collect all income and population data for each county
5. Fill in the skeleton with the income and population data
6. Output the enriched skeleton

## All GEOIDs

In [26]:
# Mapping of state abbreviations to FIPS codes
state_fips_codes = {
    'WI': '55',
    'MA': '25',
    'GA': '13',
    'NC': '37'
}
states_of_interest = state_fips_codes.keys()
state_fips_of_interest = [state_fips_codes[state] for state in states_of_interest]

# Load and filter the shapefile
gdf = gpd.read_file('../data/county_shapes/cb_2018_us_county_5m.shp')
gdf_states = gdf[gdf['STATEFP'].isin(state_fips_of_interest)]
gdf_states.reset_index(inplace = True)
fips_to_state = { code: abbr for abbr, code in state_fips_codes.items() }
gdf_states['State_Abbreviation'] = gdf_states['STATEFP'].map(fips_to_state)
gdf_states['GEOID'] = gdf_states['GEOID'].astype(str)
gdf_states.rename(columns = {'NAME' : 'County'}, inplace = True)
gdf_states = gdf_states[['County', 'State_Abbreviation', 'GEOID', 'geometry']]

/opt/homebrew/Caskroom/miniforge/base/envs/erdos_fall_2024/lib/python3.12/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/homebrew/Caskroom/miniforge/base/envs/erdos_fall_2024/lib/python3.12/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/jk/npp70g_j2fbg8x9prxslv1zw6_ptrw/T/ipykernel_39073/1855564859.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See 

## Cross with all Years

In [27]:
years = [2019, 2020, 2021, 2022]
geoids = gdf_states['GEOID'].unique()
# skeleton = pd.DataFrame([(year, geoid) for year in years for geoid in geoids], columns=['Year', 'GEOID'])
skeleton = gdf_states.merge(pd.DataFrame({'Year': years}), how = 'cross')
skeleton['PassRate'] = None
skeleton['Income'] = None
skeleton['Population'] = None
skeleton.head()

,County,State_Abbreviation,GEOID,geometry,Year,PassRate,Income,Population
0,Haralson,GA,13143,"POLYGON ((-85.38651 33.90172, -85.21279 33.899...",2019,None,None,None
1,Haralson,GA,13143,"POLYGON ((-85.38651 33.90172, -85.21279 33.899...",2020,None,None,None
2,Haralson,GA,13143,"POLYGON ((-85.38651 33.90172, -85.21279 33.899...",2021,None,None,None
3,Haralson,GA,13143,"POLYGON ((-85.38651 33.90172, -85.21279 33.899...",2022,None,None,None
4,Kewaunee,WI,55061,"POLYGON ((-87.76532 44.4145, -87.76238 44.6445...",2019,None,None,None


## Get AP Performance for each State

### Georgia

In [28]:
GA_raw = gimmeGA(prefix = prefix)
GA_raw = GA_raw[GA_raw['TEST_CMPNT_TYP_NM'] == 'ALL Subjects'][['County', 'Year', 'Pass Rate']]
GA_raw.reset_index(inplace = True)
GA_raw.rename(columns = {'Pass Rate' : 'PassRate'}, inplace = True)
GA_raw['State_Abbreviation'] = 'GA'
GA_raw['PassRate'] *= 100
GA_map_data = GA_raw[['County', 'State_Abbreviation', 'Year', 'PassRate']]
GA_map_data.head()

,County,State_Abbreviation,Year,PassRate
0,Appling,GA,2019,40.217391
1,Atkinson,GA,2019,21.621622
2,Baldwin,GA,2019,28.260870
3,Banks,GA,2019,43.243243
4,Barrow,GA,2019,46.895641


### Massachusetts

In [31]:
# MA_map_data
MA_raw = pd.read_excel(prefix + 'data/Massachusetts/county_passrate_18_22.xlsx', sheet_name='2018-22')
MA_raw.rename(columns = {'Pass Rate' : 'PassRate'}, inplace = True)
MA_raw['State_Abbreviation'] = 'MA'
MA_raw['PassRate'] *= 100
MA_map_data = MA_raw[['County', 'State_Abbreviation', 'Year', 'PassRate']]
MA_map_data

,County,State_Abbreviation,Year,PassRate
0,Barnstable,MA,2018,86.147860
1,Berkshire,MA,2018,78.299120
2,Bristol,MA,2018,88.265149
3,Essex,MA,2018,86.004178
4,Franklin,MA,2018,92.789374
...,...,...,...,...
60,Nantucket,MA,2022,87.804878
61,Norfolk,MA,2022,75.233793
62,Plymouth,MA,2022,87.764489
63,Suffolk,MA,2022,52.528883


### Wisconsin

In [30]:
WI_raw = pd.read_csv('../data/Wisconsin/Wisconsin_combined.csv')
WI_raw.rename(columns = {
    'COUNTY' : 'County',
    'PERCENT_3_OR_ABOVE' : 'PassRate'
    }, inplace = True)
rename_years = {
    '2018-19' : 2018, 
    '2019-20' : 2019, 
    '2020-21' : 2020, 
    '2021-22' : 2021, 
    '2022-23' : 2022
}
WI_raw['Year'] = WI_raw.apply(lambda row : rename_years[row['Year']], axis = 1)
WI_raw['State_Abbreviation'] = 'WI'
WI_raw['County'] = WI_raw['County'].replace('Saint Croix', 'St. Croix')
WI_map_data = WI_raw[['County', 'State_Abbreviation', 'Year', 'PassRate']]
WI_map_data.head()

,County,State_Abbreviation,Year,PassRate
0,Adams,WI,2018,28.947368
1,Ashland,WI,2018,62.500000
2,Barron,WI,2018,51.530612
3,Bayfield,WI,2018,64.285714
4,Brown,WI,2018,69.745411


## Add AP Performance to Skeleton

In [112]:
map_data = pd.concat([GA_map_data, MA_map_data, WI_map_data], ignore_index = True) 
merged = skeleton.merge(
    map_data,
    on = ['County', 'State_Abbreviation', 'Year'],
    how = 'left',
    suffixes = ('', '_map_data')
)
merged['PassRate'] = merged['PassRate'].fillna(merged['PassRate_map_data'])
merged = merged.drop(columns=['PassRate_map_data'])
skeleton = merged
skeleton.head()

/var/folders/jk/npp70g_j2fbg8x9prxslv1zw6_ptrw/T/ipykernel_3580/1059422135.py:8: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged['PassRate'] = merged['PassRate'].fillna(merged['PassRate_map_data'])


,County,State_Abbreviation,GEOID,geometry,Year,PassRate,Income,Population
0,Haralson,GA,13143,"POLYGON ((-85.38651 33.90172, -85.21279 33.899...",2019,NaN,None,None
1,Haralson,GA,13143,"POLYGON ((-85.38651 33.90172, -85.21279 33.899...",2020,NaN,None,None
2,Haralson,GA,13143,"POLYGON ((-85.38651 33.90172, -85.21279 33.899...",2021,33.333333,None,None
3,Haralson,GA,13143,"POLYGON ((-85.38651 33.90172, -85.21279 33.899...",2022,72.727273,None,None
4,Kewaunee,WI,55061,"POLYGON ((-87.76532 44.4145, -87.76238 44.6445...",2019,61.290323,None,None


## Get Income and Population Data

### Income

In [113]:
CAINC_raw = pd.read_csv(prefix + 'data/CAINC_Incomes_Counties_2019_2022.csv')
incomes = CAINC_raw[CAINC_raw['Description'] == 'Per capita personal income (dollars) 2/']
incomes = incomes[incomes['GeoName'].str.contains(',')]
def split_county_state(geoName, county = True):
    S = geoName.split(', ')
    return S[0] if county else S[len(S) - 1]
incomes['County'] = incomes.apply(lambda row: split_county_state(row['GeoName'], county = True), axis = 1)
incomes['State_Abbreviation'] = incomes.apply(lambda row: split_county_state(row['GeoName'], county = False), axis = 1)
incomes[['County', 'State_Abbreviation', '2019', '2020', '2021', '2022']]
melted = incomes.melt(
    id_vars = ['County', 'State_Abbreviation'],
    value_vars = ['2019', '2020', '2021', '2022'],
    var_name = 'Year',
    value_name = 'Income'
)
melted['Year'] = melted['Year'].astype(int)
melted['County'] = melted['County'].replace('Saint Croix', 'St. Croix')
incomes = melted
incomes.head(10)

,County,State_Abbreviation,Year,Income
0,Autauga,AL,2019,42550
1,Baldwin,AL,2019,47911
2,Barbour,AL,2019,34685
3,Bibb,AL,2019,32104
4,Blount,AL,2019,36561
5,Bullock,AL,2019,27124
6,Butler,AL,2019,37702
7,Calhoun,AL,2019,36747
8,Chambers,AL,2019,33542
9,Cherokee,AL,2019,37831


### Population

In [114]:
population = CAINC_raw[CAINC_raw['Description'] == 'Population (persons) 1/']
population = population[population['GeoName'].str.contains(',')]
def split_county_state(geoName, county = True):
    S = geoName.split(', ')
    return S[0] if county else S[len(S) - 1]
population['County'] = population.apply(lambda row: split_county_state(row['GeoName'], county = True), axis = 1)
population['State_Abbreviation'] = population.apply(lambda row: split_county_state(row['GeoName'], county = False), axis = 1)
population[['County', 'State_Abbreviation', '2019', '2020', '2021', '2022']]
melted = population.melt(
    id_vars = ['County', 'State_Abbreviation'],
    value_vars = ['2019', '2020', '2021', '2022'],
    var_name = 'Year',
    value_name = 'Population'
)
melted['Year'] = melted['Year'].astype(int)
melted['County'] = melted['County'].replace('Saint Croix', 'St. Croix')
population = melted
population.head(10)

,County,State_Abbreviation,Year,Population
0,Autauga,AL,2019,58245
1,Baldwin,AL,2019,227079
2,Barbour,AL,2019,25205
3,Bibb,AL,2019,22405
4,Blount,AL,2019,58956
5,Bullock,AL,2019,10455
6,Butler,AL,2019,19097
7,Calhoun,AL,2019,116669
8,Chambers,AL,2019,34914
9,Cherokee,AL,2019,25061


# Add Income and Population data to Skeleton

### Income

In [116]:
merged = skeleton.merge(
    incomes,
    on = ['County', 'State_Abbreviation', 'Year'],
    how = 'left',
    suffixes = ('', '_incomes')
)

merged['Income'] = merged['Income'].fillna(merged['Income_incomes'])
merged = merged.drop(columns=['Income_incomes'])
skeleton = merged
skeleton

,County,State_Abbreviation,GEOID,geometry,Year,PassRate,Income,Population
0,Haralson,GA,13143,"POLYGON ((-85.38651 33.90172, -85.21279 33.899...",2019,NaN,38099,None
1,Haralson,GA,13143,"POLYGON ((-85.38651 33.90172, -85.21279 33.899...",2020,NaN,40931,None
2,Haralson,GA,13143,"POLYGON ((-85.38651 33.90172, -85.21279 33.899...",2021,33.333333,44439,None
3,Haralson,GA,13143,"POLYGON ((-85.38651 33.90172, -85.21279 33.899...",2022,72.727273,45136,None
4,Kewaunee,WI,55061,"POLYGON ((-87.76532 44.4145, -87.76238 44.6445...",2019,61.290323,48131,None
...,...,...,...,...,...,...,...,...
975,Ashland,WI,55003,"MULTIPOLYGON (((-90.46546 47.00259, -90.45745 ...",2022,43.750000,46014,None
976,Webster,GA,13307,"POLYGON ((-84.65501 31.96153, -84.64536 31.961...",2019,NaN,37833,None
977,Webster,GA,13307,"POLYGON ((-84.65501 31.96153, -84.64536 31.961...",2020,NaN,39904,None
978,Webster,GA,13307,"POLYGON ((-84.65501 31.96153, -84.64536 31.961...",2021,NaN,40385,None


### Population

In [117]:
merged = skeleton.merge(
    population,
    on = ['County', 'State_Abbreviation', 'Year'],
    how = 'left',
    suffixes = ('', '_populations')
)

merged['Population'] = merged['Population'].fillna(merged['Population_populations'])
merged = merged.drop(columns=['Population_populations'])
skeleton = merged
skeleton

,County,State_Abbreviation,GEOID,geometry,Year,PassRate,Income,Population
0,Haralson,GA,13143,"POLYGON ((-85.38651 33.90172, -85.21279 33.899...",2019,NaN,38099,29534
1,Haralson,GA,13143,"POLYGON ((-85.38651 33.90172, -85.21279 33.899...",2020,NaN,40931,30037
2,Haralson,GA,13143,"POLYGON ((-85.38651 33.90172, -85.21279 33.899...",2021,33.333333,44439,30579
3,Haralson,GA,13143,"POLYGON ((-85.38651 33.90172, -85.21279 33.899...",2022,72.727273,45136,31337
4,Kewaunee,WI,55061,"POLYGON ((-87.76532 44.4145, -87.76238 44.6445...",2019,61.290323,48131,20596
...,...,...,...,...,...,...,...,...
975,Ashland,WI,55003,"MULTIPOLYGON (((-90.46546 47.00259, -90.45745 ...",2022,43.750000,46014,16039
976,Webster,GA,13307,"POLYGON ((-84.65501 31.96153, -84.64536 31.961...",2019,NaN,37833,2353
977,Webster,GA,13307,"POLYGON ((-84.65501 31.96153, -84.64536 31.961...",2020,NaN,39904,2352
978,Webster,GA,13307,"POLYGON ((-84.65501 31.96153, -84.64536 31.961...",2021,NaN,40385,2356


## Add to/Clean the Skeleton

In [118]:
skeleton['County_State'] = skeleton['County'] + ', ' + skeleton['State_Abbreviation']
skeleton = skeleton[['County', 'State_Abbreviation', 'Year', 'PassRate', 'Income', 'Population', 'County_State', 'GEOID', 'geometry']]
skeleton.sample(10)

,County,State_Abbreviation,Year,PassRate,Income,Population,County_State,GEOID,geometry
188,Newton,GA,2019,35.176282,34978,111445,"Newton, GA",13217,"POLYGON ((-84.04449 33.52578, -84.03212 33.539..."
689,Terrell,GA,2020,NaN,42725,9138,"Terrell, GA",13273,"POLYGON ((-84.59978 31.92017, -84.45331 31.919..."
839,Brooks,GA,2022,15.662651,39500,16253,"Brooks, GA",13027,"POLYGON ((-83.74226 30.74204, -83.74082 30.818..."
605,Langlade,WI,2020,30.158730,47949,19462,"Langlade, WI",55067,"POLYGON ((-89.42482 45.46717, -89.36405 45.469..."
967,Harris,GA,2022,69.781931,58675,36276,"Harris, GA",13145,"POLYGON ((-85.18579 32.87027, -85.10949 32.870..."
787,Paulding,GA,2022,61.964736,48143,178421,"Paulding, GA",13223,"POLYGON ((-85.04983 33.95264, -84.97868 33.951..."
113,Polk,GA,2020,33.125000,36460,42923,"Polk, GA",13233,"POLYGON ((-85.42188 34.08082, -85.28332 34.079..."
449,Worth,GA,2020,15.625000,38257,20738,"Worth, GA",13321,"POLYGON ((-84.03234 31.71677, -84.00762 31.735..."
236,McIntosh,GA,2019,NaN,39985,11204,"McIntosh, GA",13191,"POLYGON ((-81.66321 31.53867, -81.62222 31.551..."
541,Sumter,GA,2020,29.032258,40077,29516,"Sumter, GA",13261,"POLYGON ((-84.4438 31.96898, -84.43301 32.0419..."


In [119]:
skeleton.to_csv('States_Counties_Map_Data.csv', index = False)

----------

## Wisconsin 2017-2018 Predictions using Combined Model

In [2]:
# Get Wisconsin counties shape data
state_fips_codes = {
    'WI': '55'
}
states_of_interest = state_fips_codes.keys()
state_fips_of_interest = [state_fips_codes[state] for state in states_of_interest]

# Load and filter the shapefile
gdf = gpd.read_file('../data/county_shapes/cb_2018_us_county_5m.shp')
gdf_states = gdf[gdf['STATEFP'].isin(state_fips_of_interest)]
gdf_states.reset_index(inplace = True)
fips_to_state = { code: abbr for abbr, code in state_fips_codes.items() }
gdf_states['State_Abbreviation'] = gdf_states['STATEFP'].map(fips_to_state)
gdf_states['GEOID'] = gdf_states['GEOID'].astype(str)
gdf_states.rename(columns = {'NAME' : 'County'}, inplace = True)
gdf_states = gdf_states[['County', 'State_Abbreviation', 'GEOID', 'geometry']]

WI_predictions_df = pd.read_csv('../data/WI_2017_predictions.csv').drop(['Unnamed: 0'], axis = 1)
WI_predictions_df.rename(columns = {'COUNTY': 'County', 'PERCENT_3_OR_ABOVE': 'PassRate', 'MODEL_PREDICTIONS' : 'PassRate_Predicted'}, inplace = True)

/opt/homebrew/Caskroom/miniforge/base/envs/erdos_fall_2024/lib/python3.12/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/opt/homebrew/Caskroom/miniforge/base/envs/erdos_fall_2024/lib/python3.12/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/var/folders/jk/npp70g_j2fbg8x9prxslv1zw6_ptrw/T/ipykernel_92805/3274926098.py:15: SettingWithCopyWarning: 
A value 

In [3]:
WI_predictions_df

,County,PassRate,PassRate_Predicted
0,Adams,30.681818,28.125456
1,Ashland,51.612903,55.818817
2,Barron,49.769585,52.217087
3,Bayfield,76.470588,60.077816
4,Brown,70.513192,66.667720
...,...,...,...
65,Waukesha,72.446982,73.019180
66,Waupaca,47.780679,50.439266
67,Waushara,55.319149,43.146534
68,Winnebago,66.432337,66.845856


In [4]:
merged = gdf_states.merge(
    WI_predictions_df,
    on = ['County'],
    how = 'left',
    suffixes = ('', '_merged')
)
merged = merged[['County', 'PassRate', 'PassRate_Predicted', 'GEOID', 'geometry']]
merged.sample(5)

,County,PassRate,PassRate_Predicted,GEOID,geometry
20,Sawyer,46.666667,45.87826,55113,"POLYGON ((-91.55095 46.04111, -91.55128 46.157..."
64,Dane,73.846488,65.77051,55025,"POLYGON ((-89.83823 43.0747, -89.83796 43.1185..."
58,Burnett,56.034483,43.26537,55013,"POLYGON ((-92.88571 45.64602, -92.88399 45.654..."
16,Dunn,69.058296,66.45289,55033,"POLYGON ((-92.15646 45.20955, -92.04246 45.208..."
4,Menominee,NaN,NaN,55078,"POLYGON ((-88.98218 45.11773, -88.93055 45.117..."


In [5]:
merged.to_csv('Wisconsin_2017_predictions_and_shapes.csv', index = False)

In [6]:
merged

,County,PassRate,PassRate_Predicted,GEOID,geometry
0,Kewaunee,63.559322,56.555380,55061,"POLYGON ((-87.76532 44.4145, -87.76238 44.6445..."
1,Marathon,65.634921,63.812794,55073,"POLYGON ((-90.31657 44.95328, -90.31504 45.033..."
2,Rusk,69.354839,36.437366,55107,"POLYGON ((-91.54029 45.6376, -90.67875 45.6382..."
3,Taylor,57.971014,53.694080,55119,"POLYGON ((-90.92522 45.29206, -90.92534 45.379..."
4,Menominee,NaN,NaN,55078,"POLYGON ((-88.98218 45.11773, -88.93055 45.117..."
...,...,...,...,...,...
67,Pepin,54.945055,59.214120,55091,"POLYGON ((-92.31623 44.54097, -92.1353 44.5395..."
68,Lincoln,68.776371,58.474900,55069,"POLYGON ((-90.04614 45.34031, -90.04227 45.381..."
69,Juneau,43.877551,40.334667,55057,"POLYGON ((-90.31269 43.98138, -90.31252 44.155..."
70,Brown,70.513192,66.667720,55009,"POLYGON ((-88.25256 44.67981, -88.24269 44.679..."
